In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd

In [3]:
import json
import itertools

from pathlib import Path

In [4]:
from mppsteel.config.model_scenarios import DEFAULT_SCENARIO
from mppsteel.config.model_config import USD_TO_EUR_CONVERSION_DEFAULT

from mppsteel.config.reference_lists import RESOURCE_CATEGORY_MAPPER

from mppsteel.data_preprocessing.variable_plant_cost_archetypes import (
    plant_variable_costs,
    generate_feedstock_dict,
    generate_variable_costs,
    vc_mapper,
)

from mppsteel.utility.file_handling_utility import (
    read_pickle_folder,
    get_scenario_pkl_path,
)

from mppsteel.config.model_config import (
    PKL_DATA_FORMATTED,
    PKL_DATA_IMPORTS,
    MODEL_YEAR_RANGE,
)

In [5]:
scenario_dict = DEFAULT_SCENARIO.copy()
scenario_dict["usd_to_eur"] = USD_TO_EUR_CONVERSION_DEFAULT
scenario_dict["eur_to_usd"] = 1.0 / scenario_dict["usd_to_eur"]

In [6]:
%%time
df_reference = plant_variable_costs(scenario_dict)

Variable Cost Loop: 100%|██████████████████████████████████████████████████████| 2356/2356 [02:05<00:00, 18.83it/s]


CPU times: user 2min 5s, sys: 1.47 s, total: 2min 6s
Wall time: 2min 6s


In [7]:
intermediate_path = get_scenario_pkl_path(
    scenario_dict["scenario_name"], "intermediate"
)
eur_to_usd_rate = scenario_dict["eur_to_usd"]

steel_plants = read_pickle_folder(PKL_DATA_FORMATTED, "steel_plants_processed", "df")
steel_plant_region_ng_dict = (
    steel_plants[["country_code", "cheap_natural_gas"]]
    .set_index("country_code")
    .to_dict()["cheap_natural_gas"]
)
power_grid_prices_ref = read_pickle_folder(
    intermediate_path, "power_grid_prices_ref", "df"
)
h2_prices_ref = read_pickle_folder(intermediate_path, "h2_prices_ref", "df")
bio_model_prices_ref = read_pickle_folder(
    intermediate_path, "bio_model_prices_ref", "df"
)
ccs_model_transport_ref = read_pickle_folder(
    intermediate_path, "ccs_model_transport_ref", "df"
)
ccs_model_storage_ref = read_pickle_folder(
    intermediate_path, "ccs_model_storage_ref", "df"
)
business_cases = read_pickle_folder(
    PKL_DATA_FORMATTED, "standardised_business_cases", "df"
).reset_index()
static_energy_prices = read_pickle_folder(
    PKL_DATA_IMPORTS, "static_energy_prices", "df"
)[["Metric", "Year", "Value"]]
static_energy_prices.set_index(["Metric", "Year"], inplace=True)
feedstock_dict = generate_feedstock_dict(eur_to_usd_rate)
steel_plant_country_codes = list(steel_plants["country_code"].unique())
product_range_year_country = list(
    itertools.product(MODEL_YEAR_RANGE, steel_plant_country_codes)
)

# Build Price Lookup DataFrames From Dicts

## Helper Functions

In [8]:
def create_df_from_years_and_contry_codes(year_range, country_codes):
    return pd.merge(
        pd.DataFrame(country_codes, columns=["country_code"]),
        pd.DataFrame(year_range, columns=["year"]),
        how="cross",
    )

## Power Grid Prices

In [9]:
%%time
pgp_ref_list = [
    (year, cc, price) for (year, cc), price in power_grid_prices_ref.items()
]
df_pgp = pd.DataFrame(pgp_ref_list, columns=("year", "country_code", "price"))
df_pgp["material_category"] = "Electricity"
# df_pgp["country_code"] = df_pgp["country_code"].astype("category")

CPU times: user 5.93 ms, sys: 852 µs, total: 6.78 ms
Wall time: 6.01 ms


## Hydrogen Prices

In [10]:
%%time
h2_ref_list = [(year, cc, price) for (year, cc), price in h2_prices_ref.items()]
df_hydrogen = pd.DataFrame(h2_ref_list, columns=("year", "country_code", "price"))
df_hydrogen["material_category"] = "Hydrogen"

CPU times: user 4.95 ms, sys: 582 µs, total: 5.54 ms
Wall time: 4.9 ms


## Bio Model Prices

In [11]:
%%time
df_bio_mass = pd.DataFrame(
    (
        (year, country_code, price)
        for (year, country_code), price in bio_model_prices_ref.items()
    ),
    columns=("year", "country_code", "price"),
)
df_bio_mass["material_category"] = "Biomass"
df_bio_methane = df_bio_mass.copy()
df_bio_methane["material_category"] = "Biomethane"

CPU times: user 8.23 ms, sys: 1.83 ms, total: 10.1 ms
Wall time: 8.52 ms


## CCS Transport / Storage Prices

In [12]:
%%time
df_year = pd.DataFrame(MODEL_YEAR_RANGE, columns=["year"])
df_storage = pd.DataFrame(
    ccs_model_storage_ref.items(), columns=("country_code", "price_storage")
)
df_transport = pd.DataFrame(
    ccs_model_transport_ref.items(), columns=("country_code", "price_transport")
)
df_store_trans = pd.merge(df_storage, df_transport, on=("country_code"))
df_store_trans["price"] = df_store_trans.price_storage + df_store_trans.price_transport
df_store_trans = df_store_trans.drop(["price_storage", "price_transport"], axis=1)
df_store_trans = df_store_trans.merge(df_year, how="cross")
df_store_trans_captured = df_store_trans.copy()
df_store_trans_captured["material_category"] = "Captured CO2"
df_store_trans_used = df_store_trans.copy()
df_store_trans_used["material_category"] = "Used CO2"

CPU times: user 10.6 ms, sys: 2.08 ms, total: 12.7 ms
Wall time: 11.1 ms


## Static Energy Prices

In [15]:
%%time
low_natural_gas_cc = [
    country_code
    for country_code, flag in steel_plant_region_ng_dict.items()
    if flag == 1
]
high_natural_gas_cc = [
    country_code
    for country_code, flag in steel_plant_region_ng_dict.items()
    if flag == 0
]

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 18.1 µs


In [16]:
def get_gas_prices_per_country_and_year(static_energy_prices, gas_type, country_codes):
    gas_prices = static_energy_prices.loc[gas_type].reset_index().copy()
    year_to_price = dict(zip(gas_prices.Year.values, gas_prices.Value.values))
    default_price = year_to_price[
        2026
    ]  # FIXME make this depend on the available range of years
    df_data = {"year": [], "price": []}
    for year in MODEL_YEAR_RANGE:
        df_data["year"].append(year)
        df_data["price"].append(year_to_price.get(year, default_price))
    df = pd.DataFrame(df_data)
    return pd.merge(
        df, pd.DataFrame(country_codes, columns=["country_code"]), how="cross"
    )

In [17]:
%%time
df_gas_low = get_gas_prices_per_country_and_year(
    static_energy_prices, "Natural gas - low", low_natural_gas_cc
)
df_gas_low["material_category"] = "Natural gas"
df_gas_high = get_gas_prices_per_country_and_year(
    static_energy_prices, "Natural gas - high", high_natural_gas_cc
)
df_gas_high["material_category"] = "Natural gas"

CPU times: user 13.9 ms, sys: 2.33 ms, total: 16.3 ms
Wall time: 14.2 ms


## Plastic Waste Prices

In [18]:
%%time
df_waste = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_waste["price"] = feedstock_dict["Plastic waste"]
df_waste["material_category"] = "Plastic waste"

CPU times: user 6.49 ms, sys: 1.8 ms, total: 8.3 ms
Wall time: 6.7 ms


## Fossil Fuel Prices

In [19]:
fossil_categories = [
    k
    for k, v in RESOURCE_CATEGORY_MAPPER.items()
    if v == "Fossil Fuels" and k not in ("Natural gas", "Plastic waste")
]

In [20]:
fossil_categories

['Met coal', 'Thermal coal', 'Coke', 'COG', 'BF gas', 'BOF gas']

In [21]:
def get_fossil_category_prices(static_energy_prices, category, country_codes):
    fossil_prices = static_energy_prices.loc[category].reset_index().copy()
    year_to_price = dict(zip(fossil_prices.Year.values, fossil_prices.Value.values))
    default_price = year_to_price[
        2026
    ]  # FIXME make this depend on the available range of years
    data = {"year": [], "price": []}
    for year in MODEL_YEAR_RANGE:
        data["year"].append(year)
        data["price"].append(year_to_price.get(year, default_price))
    df = pd.DataFrame(data)
    df["material_category"] = category
    return pd.merge(
        df, pd.DataFrame(country_codes, columns=["country_code"]), how="cross"
    )

In [22]:
%%time
fossil_price_dfs = []
for category in fossil_categories:
    fossil_price_dfs.append(
        get_fossil_category_prices(
            static_energy_prices, category, steel_plant_country_codes
        )
    )

CPU times: user 35.3 ms, sys: 2.38 ms, total: 37.7 ms
Wall time: 35.7 ms


## Feedstock / Raw Material

In [23]:
%%time
df_iron_ore = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_iron_ore["price"] = feedstock_dict["Iron ore"]
df_iron_ore["material_category"] = "Iron ore"
df_scrap = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_scrap["price"] = feedstock_dict["Scrap"]
df_scrap["material_category"] = "Scrap"
df_dri = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_dri["price"] = feedstock_dict["DRI"]
df_dri["material_category"] = "DRI"

CPU times: user 14.9 ms, sys: 1.94 ms, total: 16.9 ms
Wall time: 15.2 ms


## Other Opex

In [24]:
%%time
df_bf_slag = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_bf_slag["price"] = feedstock_dict["BF slag"]
df_bf_slag["material_category"] = "BF slag"
df_other_slag = create_df_from_years_and_contry_codes(MODEL_YEAR_RANGE, steel_plant_country_codes)
df_other_slag["price"] = feedstock_dict["Other slag"]
df_other_slag["material_category"] = "Other slag"

CPU times: user 11.3 ms, sys: 1.99 ms, total: 13.3 ms
Wall time: 11.5 ms


In [25]:
%%time
df_steam = get_gas_prices_per_country_and_year(
    static_energy_prices, "Steam", steel_plant_country_codes
)
df_steam["material_category"] = "Steam"

CPU times: user 7.88 ms, sys: 1.95 ms, total: 9.82 ms
Wall time: 8.03 ms


In [26]:
[(k, v) for k, v in RESOURCE_CATEGORY_MAPPER.items() if v == "Other Opex"]

[('Steam', 'Other Opex'),
 ('BF slag', 'Other Opex'),
 ('Other slag', 'Other Opex')]

## Merge Price DataFrames

In [27]:
%%time

df_prices = pd.concat([
    df_pgp, df_hydrogen, df_bio_mass, df_bio_methane, df_store_trans_captured, df_store_trans_used, df_gas_low, df_gas_high,
    df_waste, *fossil_price_dfs, df_iron_ore, df_scrap, df_dri, df_bf_slag, df_other_slag, df_steam
])
df_prices["country_code"] = df_prices["country_code"].astype("category")

CPU times: user 17.9 ms, sys: 2.84 ms, total: 20.7 ms
Wall time: 19.1 ms


In [28]:
df_prices.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79174 entries, 0 to 2355
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   year               79174 non-null  int64   
 1   country_code       79174 non-null  category
 2   price              79174 non-null  float64 
 3   material_category  79174 non-null  object  
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 6.9 MB


# Base DataFrame

In [29]:
%%time
dyc = pd.DataFrame(product_range_year_country, columns=("year", "country_code"))
emissions = set([k for k, v in RESOURCE_CATEGORY_MAPPER.items() if v == "Emissivity"])
db = business_cases.copy()
db = db[~db.material_category.isin(emissions)]
df = db.merge(dyc, how="cross")
not_categorical = set(["year", "value"])
categorical_columns = [col for col in df.columns if col not in not_categorical]
for col in categorical_columns:
    df[col] = df[col].astype("category")
df["cost"] = 0.0
# df = df.reset_index().set_index(["material_category", "index"])

CPU times: user 454 ms, sys: 40.3 ms, total: 494 ms
Wall time: 493 ms


In [30]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088472 entries, 0 to 1088471
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype   
---  ------             --------------    -----   
 0   technology         1088472 non-null  category
 1   material_category  1088472 non-null  category
 2   metric_type        1088472 non-null  category
 3   unit               1088472 non-null  category
 4   value              1088472 non-null  float64 
 5   year               1088472 non-null  int64   
 6   country_code       1088472 non-null  category
 7   cost               1088472 non-null  float64 
dtypes: category(5), float64(2), int64(1)
memory usage: 38.4 MB


In [31]:
%%time
# dm = (
#     df.reset_index()
#     .merge(df_prices, on=("material_category", "year", "country_code"))
#     .set_index("index")
# )
# dm["cost"] = dm.value * dm.price
# df["cost"] = dm.cost

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [32]:
# query_str = "material_category == 'Electricity' and technology == 'DRI-Melt-BOF+CCUS' and year == 2050 and country_code == 'USA'"
# df.query(query_str)

In [33]:
%%time
dm = df.merge(df_prices, on=("material_category", "year", "country_code"), how="left")
dm["cost"] = dm.value * dm.price
dm["cost"] = dm.cost.fillna(0.0)
# df["cost"] = dm.cost

CPU times: user 276 ms, sys: 44.9 ms, total: 321 ms
Wall time: 320 ms


In [34]:
query_str = "material_category == 'Electricity' and technology == 'DRI-Melt-BOF+CCUS' and year == 2050 and country_code == 'USA'"
dm.query(query_str)

,technology,material_category,metric_type,unit,value,year,country_code,cost,price
925907,DRI-Melt-BOF+CCUS,Electricity,Purchased energy,GJ/t steel,3.50132,2050,USA,59.091243,16.87685


In [35]:
# def calculate_price_for_electricity(df, df_pgp):
#     de = df[df.material_category == "Electricity"]
#     dm = de.reset_index().merge(df_pgp, on=("year", "country_code")).set_index("index")
#     dm["cost"] = dm.value * dm.price
#     df["cost"] = dm.cost
#     return df

In [36]:
emissions = set([k for k, v in RESOURCE_CATEGORY_MAPPER.items() if v == "Emissivity"])

# Compare Optimized Version with Reference

In [37]:
df_reference.shape

(1088472, 9)

In [38]:
dm.shape

(1088472, 9)

In [39]:
df.shape

(1088472, 8)

In [40]:
df_reference.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088472 entries, 0 to 1192135
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   technology         1088472 non-null  object 
 1   material_category  1088472 non-null  object 
 2   metric_type        1088472 non-null  object 
 3   unit               1088472 non-null  object 
 4   value              1088472 non-null  float64
 5   year               1088472 non-null  int64  
 6   country_code       1088472 non-null  object 
 7   cost               1088472 non-null  float64
 8   cost_type          1088472 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 443.3 MB


In [41]:
df_reference.material_category.unique()

array(['Iron ore', 'Scrap', 'DRI', 'Met coal', 'Coke', 'Thermal coal',
       'BF gas', 'COG', 'BOF gas', 'Natural gas', 'Plastic waste',
       'Biomass', 'Biomethane', 'Hydrogen', 'Electricity', 'Steam',
       'BF slag', 'Other slag', 'Captured CO2', 'Used CO2', 'Emissivity'],
      dtype=object)

In [42]:
def build_pk(df):
    df["year"] = df["year"].astype("str")
    pk_columns = ["material_category", "technology", "country_code", "year"]
    df["pk"] = df[pk_columns].agg("_".join, axis=1)
    return df.drop(pk_columns, axis=1).set_index("pk").sort_index()


def build_comparable_dataframes(category, df_reference, df_optimized):
    dfr = (
        df_reference[df_reference.material_category == category]
        .copy()
        .drop("cost_type", axis=1)
        .reset_index(drop=True)
    )
    dmo = (
        df_optimized[df_optimized.material_category == category]
        .copy()
        .drop("price", axis=1)
        .reset_index(drop=True)
    )
    return build_pk(dfr), build_pk(dmo)


def compare_by_material_category(category, df_reference, df_optimized):
    dfr, dmo = build_comparable_dataframes(category, df_reference, df_optimized)
    print("Category: ", category)
    print("shape: ", dfr.shape, dmo.shape, dfr.shape == dmo.shape)
    print("column values: ", np.all((dfr == dmo).all().values))
    print("sum cost: ", dfr.cost.sum(), dmo.cost.sum())
    print("-----------------------------")

In [43]:
compare_by_material_category("Hydrogen", df_reference, dm)

Category:  Hydrogen
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  1149369.2660721818 1149369.2660721818
-----------------------------


In [44]:
compare_by_material_category("Iron ore", df_reference, dm)

Category:  Iron ore
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  6224864.016437758 6224864.016437758
-----------------------------


In [45]:
# RESOURCE_CATEGORY_MAPPER

In [46]:
dfr, dmo = build_comparable_dataframes("Emissivity", df_reference, dm)

In [47]:
dfr.head()

,metric_type,unit,value,cost
pk,,,,
Emissivity_Avg BF-BOF_AGO_2020,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2021,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2022,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2023,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2024,Other,tCO2/t steel,2.561582,0.0


In [48]:
dmo.head()

,metric_type,unit,value,cost
pk,,,,
Emissivity_Avg BF-BOF_AGO_2020,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2021,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2022,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2023,Other,tCO2/t steel,2.561582,0.0
Emissivity_Avg BF-BOF_AGO_2024,Other,tCO2/t steel,2.561582,0.0


In [49]:
for category in df_reference.material_category.unique():
    compare_by_material_category(category, df_reference, dm)

Category:  Iron ore
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  6224864.016437758 6224864.016437758
-----------------------------
Category:  Scrap
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  2309189.8733712 2309189.8733712
-----------------------------
Category:  DRI
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  0.0 0.0
-----------------------------
Category:  Met coal
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  563174.7219761774 563174.7219761774
-----------------------------
Category:  Coke
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  0.0 0.0
-----------------------------
Category:  Thermal coal
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  284071.0588329466 284071.0588329466
-----------------------------
Category:  BF gas
shape:  (51832, 4) (51832, 4) True
column values:  True
sum cost:  0.0 0.0
-----------------------------
Category:  COG
shape:  (518

In [133]:
df_reference.material_category.unique().shape[0] * 51832

1088472

In [134]:
df_reference.shape

(1088472, 9)

In [146]:
df_reference[df_reference.material_category == "Emissivity"]

,technology,material_category,metric_type,unit,value,year,country_code,cost,cost_type
22,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,DZA,0.0,Carbon Cost
45,BAT BF-BOF,Emissivity,Other,tCO2/t steel,2.004292,2020,DZA,0.0,Carbon Cost
68,BAT BF-BOF_bio PCI,Emissivity,Other,tCO2/t steel,1.628892,2020,DZA,0.0,Carbon Cost
91,BAT BF-BOF_H2 PCI,Emissivity,Other,tCO2/t steel,1.822237,2020,DZA,0.0,Carbon Cost
114,BAT BF-BOF+CCUS,Emissivity,Other,tCO2/t steel,0.185017,2020,DZA,0.0,Carbon Cost
...,...,...,...,...,...,...,...,...,...
1192043,DRI-Melt-BOF,Emissivity,Other,tCO2/t steel,0.782344,2050,USA,0.0,Carbon Cost
1192066,DRI-Melt-BOF+CCUS,Emissivity,Other,tCO2/t steel,0.149903,2050,USA,0.0,Carbon Cost
1192089,DRI-Melt-BOF_100% zero-C H2,Emissivity,Other,tCO2/t steel,0.051255,2050,USA,0.0,Carbon Cost
1192112,Electrowinning-EAF,Emissivity,Other,tCO2/t steel,0.081600,2050,USA,0.0,Carbon Cost


In [147]:
dm[dm.material_category == "Emissivity"]

,technology,material_category,metric_type,unit,value,year,country_code,cost,price
47120,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,DZA,NaN,NaN
47121,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,AGO,NaN,NaN
47122,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,EGY,NaN,NaN
47123,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,LBY,NaN,NaN
47124,Avg BF-BOF,Emissivity,Other,tCO2/t steel,2.561582,2020,MAR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1088467,BAT BF-BOF+BECCUS,Emissivity,Other,tCO2/t steel,-0.404088,2050,SAU,NaN,NaN
1088468,BAT BF-BOF+BECCUS,Emissivity,Other,tCO2/t steel,-0.404088,2050,SYR,NaN,NaN
1088469,BAT BF-BOF+BECCUS,Emissivity,Other,tCO2/t steel,-0.404088,2050,CAN,NaN,NaN
1088470,BAT BF-BOF+BECCUS,Emissivity,Other,tCO2/t steel,-0.404088,2050,MEX,NaN,NaN
